# Ayushmitra - Unified Backend Server
**Filename:** `ayushmitra_unified_backend.ipynb`  
**Author / Team:** Ayushmitra (Pushkar Kumar & team)  
**Purpose:** This single notebook contains the complete backend for the Ayushmitra prototype. It loads all specialist ML models, integrates them with a conversational LangChain agent powered by Groq, and exposes the entire system as a single, robust FastAPI endpoint.

In [1]:
# Step 1: Install all required libraries
!pip install fastapi "uvicorn[standard]" pyngrok joblib scikit-learn tensorflow pandas nest-asyncio langchain langchain-groq requests -q
!npm install -g localtunnel

# Step 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 3: Verify the base models path
BASE_MODELS_PATH = "/content/drive/MyDrive/SIH_2025/Ayushmitra_Models" # Corrected path
import os
if not os.path.exists(BASE_MODELS_PATH):
    raise FileNotFoundError(f"The models directory does not exist. Please check the path: {BASE_MODELS_PATH}")
else:
    print("✅ Google Drive mounted and base models directory found.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.2/452.2 kB 25.3 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 3s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸Mounted at /content/drive
✅ Google Drive mounted and base models directory found.


# Step 2: Configure `ngrok` and `Groq` API Keys
This cell will securely configure both API keys needed for the application to run.

1.  Click the **key icon (🔑)** in the Colab sidebar.
2.  Create a secret named `NGROK_AUTHTOKEN` and paste your ngrok token.
3.  Create another secret named `GROQ_API_KEY` and paste your Groq API token.
4.  Ensure "Notebook access" is toggled on for both.

In [2]:
from pyngrok import ngrok
from google.colab import userdata
import os

# Configure ngrok
try:
    NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    print("✅ ngrok authtoken configured.")
except:
    print("🚨 Could not find NGROK_AUTHTOKEN in Colab secrets.")

# Configure Groq
try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY
    print("✅ Groq API Key configured.")
except:
    print("🚨 Could not find GROQ_API_KEY in Colab secrets.")

✅ ngrok authtoken configured.
✅ Groq API Key configured.


# Step 3: The Model Service Utility Class
This modular class handles loading, managing, and running predictions for each of your individual disease models.

In [3]:
import joblib
import json
import tensorflow as tf
import pandas as pd
import numpy as np

class ModelService:
    """A service to load, hold, and use all artifacts for a single ML model."""
    def __init__(self, model_dir_path):
        if not os.path.exists(model_dir_path): raise FileNotFoundError(f"Model directory not found: {model_dir_path}")
        self.model_dir, self.model_name = model_dir_path, os.path.basename(model_dir_path)
        self.model, self.scaler, self.feature_encoders, self.feature_list, self.target_encoder = None, None, {}, [], None
        self._load_artifacts()

    def _load_artifacts(self):
        print(f"--- Loading artifacts for: {self.model_name} ---")
        for filename in os.listdir(self.model_dir):
            file_path = os.path.join(self.model_dir, filename)
            try:
                if filename.endswith(('.pkl', '.joblib')):
                    if 'scaler' in filename: self.scaler = joblib.load(file_path)
                    elif 'target_label_encoder' in filename or ('label_encoder' in filename and 'feature' not in filename): self.target_encoder = joblib.load(file_path)
                    elif 'feature_encoders' in filename: self.feature_encoders = joblib.load(file_path)
                    elif 'lr_' in filename or 'rf_' in filename: self.model = joblib.load(file_path)
                elif filename.endswith('.h5'): self.model = tf.keras.models.load_model(file_path, compile=False)
                elif 'feature_list' in filename:
                    with open(file_path, 'r') as f: self.feature_list = json.load(f)
            except Exception as e: print(f"⚠️ Warning: Failed to load {filename} for {self.model_name}: {e}")

    def predict(self, raw_input: dict):
        if not all([self.model, self.feature_list, self.target_encoder]): raise Exception(f"Model '{self.model_name}' is not fully loaded.")
        df_row = pd.DataFrame([raw_input], columns=self.feature_list)
        for col, encoder in self.feature_encoders.items():
            if col in df_row.columns:
                try:
                    value_to_encode = str(df_row[col].iloc[0]).strip().capitalize()
                    df_row[col] = encoder.transform([value_to_encode])[0]
                except Exception: df_row[col] = -1
        df_row = df_row.apply(pd.to_numeric, errors='coerce').fillna(0)
        if self.scaler:
            scaled_features = self.scaler.get_feature_names_out() if hasattr(self.scaler, 'get_feature_names_out') else self.feature_list
            df_row[scaled_features] = self.scaler.transform(df_row[scaled_features])
        if isinstance(self.model, tf.keras.Model):
            pred_proba = self.model.predict(df_row.values)[0]
            pred_index = np.argmax(pred_proba)
            confidence = pred_proba[pred_index]
        else:
            pred_proba = self.model.predict_proba(df_row.values)[0]
            pred_index = np.argmax(pred_proba)
            confidence = pred_proba[pred_index]
        prediction_label = self.target_encoder.inverse_transform([pred_index])[0]
        return {"prediction": prediction_label, "confidence": float(confidence), "model_used": self.model_name}

# Step 4: Initialize All Specialist Models
This cell iterates through your model directories in Google Drive, creating a `ModelService` instance for each and storing them in a central dictionary.

In [4]:
MODELS = {}
MODEL_FOLDERS = [
    'dengue_classifier', 'fever_medicine_classifier', 'liver_classifier', 'vector_borne_classifier','fever_classifier_typhoid'
]
print("🚀 Initializing all specialist models...")
for folder in MODEL_FOLDERS:
    try:
        model_path = os.path.join(BASE_MODELS_PATH, folder)
        MODELS[folder] = ModelService(model_path)
    except Exception as e:
        print(f"🚨 FAILED to load model from folder '{folder}': {e}")
print("\n✅ All available models initialized.")

🚀 Initializing all specialist models...
--- Loading artifacts for: dengue_classifier ---
--- Loading artifacts for: fever_medicine_classifier ---
--- Loading artifacts for: liver_classifier ---
--- Loading artifacts for: vector_borne_classifier ---
--- Loading artifacts for: fever_classifier_typhoid ---

✅ All available models initialized.


# Step 5: Assemble the LangChain Conversational Agent
Here, we build the "brain" of our application. We define tools that call our specialist models directly, initialize the high-speed Groq LLM, and create the agent that can reason about when to use which tool.

In [5]:
from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
import json

# ==============================================================================
# TOOL DEFINITIONS: One function for each of your 5 specialist models
# ==============================================================================

@tool
def check_liver_condition(age: int, gender: str, total_bilirubin: float, direct_bilirubin: float, alkphos: int, sgpt: int, sgot: int, total_proteins: float, albumin: float, ag_ratio: float) -> str:
    """
    Useful for when a user wants to check for potential liver disease.
    You must provide ALL of the following patient clinical data: age, gender, total_bilirubin (TB), direct_bilirubin (DB), alkphos, sgpt, sgot, total_proteins (TP), albumin (ALB), and ag_ratio.
    """
    features = {
        "Age": age, "Gender": gender, "TB": total_bilirubin, "DB": direct_bilirubin, "Alkphos": alkphos,
        "Sgpt": sgpt, "Sgot": sgot, "TP": total_proteins, "ALB": albumin, "A/G Ratio": ag_ratio
    }
    print(f"--- TOOL CALL: Calling Liver Model with features: {features} ---")
    prediction = MODELS['liver_classifier'].predict(features)
    return json.dumps(prediction)

@tool
def recommend_fever_medicine(temperature: float, fever_severity: str, age: int, gender: str, bmi: float, headache: str, body_ache: str, fatigue: str, chronic_conditions: str, allergies: str, smoking_history: str, alcohol_consumption: str, humidity: float, aqi: int, physical_activity: str, diet_type: str, heart_rate: int, blood_pressure: str, previous_medication: str) -> str:
    """
    Useful for recommending a common fever medication (Paracetamol or Ibuprofen) based on a patient's full clinical and lifestyle profile.
    You must provide ALL 19 features to use this tool.
    """
    features = {
        'Temperature': temperature, 'Fever_Severity': fever_severity, 'Age': age, 'Gender': gender,
        'BMI': bmi, 'Headache': headache, 'Body_Ache': body_ache, 'Fatigue': fatigue,
        'Chronic_Conditions': chronic_conditions, 'Allergies': allergies, 'Smoking_History': smoking_history,
        'Alcohol_Consumption': alcohol_consumption, 'Humidity': humidity, 'AQI': aqi,
        'Physical_Activity': physical_activity, 'Diet_Type': diet_type,
        'Heart_Rate': heart_rate, 'Blood_Pressure': blood_pressure, 'Previous_Medication': previous_medication
    }
    print(f"--- TOOL CALL: Calling Fever Medicine Model ---")
    prediction = MODELS['fever_medicine_classifier'].predict(features)
    return json.dumps(prediction)

@tool
def check_dengue_symptoms(fever: int, headache: int, joint_pain: int, bleeding: int) -> str:
    """
    Useful for predicting the likelihood of Dengue fever based on key symptoms.
    You must provide all features: fever, headache, joint_pain, and bleeding. Use 1 for 'Yes' and 0 for 'No'.
    """
    features = {"Fever": fever, "Headache": headache, "JointPain": joint_pain, "Bleeding": bleeding}
    print(f"--- TOOL CALL: Calling Dengue Model with features: {features} ---")
    prediction = MODELS['dengue_classifier'].predict(features)
    return json.dumps(prediction)

@tool
def check_vector_borne_disease(patient_data: dict) -> str:
    """
    Useful for a general differential diagnosis of vector-borne diseases like Malaria, Dengue, Yellow Fever, or Typhoid based on a wide range of clinical features.
    The input must be a dictionary named 'patient_data' containing all the required clinical features.
    """
    print(f"--- TOOL CALL: Calling Vector Borne Model with {len(patient_data)} features ---")
    prediction = MODELS['vector_borne_classifier'].predict(patient_data)
    return json.dumps(prediction)

# --- NEWLY ADDED TOOL FOR TYPHOID FEVER ---
@tool
def check_typhoid_fever(age: int, gender: str, to: int, th: int, ah: int, bh: int, oxk: int, ox2: int, ox19: int) -> str:
    """
    Useful for predicting the likelihood of Typhoid fever based on serological test results from Widal (TO, TH, AH, BH) and Weil-Felix (OXK, OX2, OX19) tests.
    You must provide the patient's age, gender, and all nine test results.
    """
    features = {"Age": age, "Gender": gender, "TO": to, "TH": th, "AH": ah, "BH": bh, "OXK": oxk, "OX2": ox2, "OX19": ox19}
    print(f"--- TOOL CALL: Calling Typhoid Fever Model with features: {features} ---")
    prediction = MODELS['fever_classifier_typhoid'].predict(features)
    return json.dumps(prediction)


# ==============================================================================
# AGENT ASSEMBLY: Combine the LLM and the tools into a reasoning agent
# ==============================================================================

# Create the list of ALL 5 tools the agent can use
tools = [
    check_liver_condition,
    recommend_fever_medicine,
    check_dengue_symptoms,
    check_vector_borne_disease,
    check_typhoid_fever  # <-- Added the 5th tool
]

# --- Initialize LLM and Agent ---
llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are Ayushmitra, a helpful medical assistant. Your job is to talk to users in a friendly, conversational way to understand their health concerns. If a concern matches one of your available tools, you must first ask clear, follow-up questions to collect all the necessary information required by that tool. Once you have gathered every required piece of information, you can call the tool to get a preliminary prediction. Never make up information or give a diagnosis. If you cannot help, politely say so."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print(f"🤖 Ayushmitra Conversational Agent is ready with {len(tools)} tools.")

🤖 Ayushmitra Conversational Agent is ready with 5 tools.


# Step 6: Create and Launch the FastAPI Server
This final cell defines our API endpoint (`/ask_agent`) which takes a user's text and passes it to our LangChain agent. It then launches the server using `uvicorn` and creates a public URL with `ngrok`.

**ACTION: Run this cell and leave it running. This is your live backend.**

In [6]:
# (This is the NEW and complete code for your final server launch cell)

from fastapi import FastAPI
import nest_asyncio
import uvicorn
import subprocess

# --- Create a main.py file for the server to run ---
# This saves our FastAPI app object into a file that uvicorn can use.
# NOTE: Make sure your FastAPI app object in Cell 10 is named 'app'.
with open("main.py", "w") as f:
    f.write("""
import os
import json
import joblib
import tensorflow as tf
import pandas as pd
import numpy as np
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from langchain_groq import ChatGroq
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import tool
from fastapi.middleware.cors import CORSMiddleware

# This is a simplified re-creation of your setup.
# In a real production app, this would be structured differently.

app = FastAPI(title="Ayushmitra Unified Backend")

# Add CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Placeholder for the agent_executor. In the actual notebook,
# the real one will be available in memory.
# This file is just a scaffold for uvicorn.
class AgentQuery(BaseModel):
    text: str

@app.post("/ask_agent")
async def ask_agent_endpoint(query: AgentQuery):
    # This is a placeholder since the real agent is in the notebook's memory.
    # The actual logic is handled by the running 'app' object.
    return {"response": "This is a placeholder response from main.py"}

# The actual 'app' object from your notebook will be used by uvicorn,
# overriding this placeholder file logic.
""")

# --- Launch the server in the background ---
# We use nohup and & to make it run without blocking the cell
!nohup uvicorn main:app --host 0.0.0.0 --port 8000 &

# Give the server a few seconds to start up
import time
time.sleep(5)

# --- Start localtunnel to create the public URL ---
!lt --port 8000

nohup: appending output to 'nohup.out'
your url is: https://clever-frogs-wonder.loca.lt
^C


# Step 7: Testing the Live API Endpoint
Once the server cell above is running, you can run this cell to test it. This simulates your React frontend making a call to the live backend.

In [17]:
import requests
import json

# The script will automatically get the public URL for you
try:
    BOT_URL = ngrok.get_tunnels()[0].public_url
    print(f"Testing bot at URL: {BOT_URL}")
except Exception as e:
    print(f"🚨 Could not get ngrok URL. Make sure the server cell is running. Error: {e}")
    BOT_URL = None

def test_api(text):
    if not BOT_URL: return
    print(f"--- Sending query to API: '{text}' ---")
    try:
        response = requests.post(f"{BOT_URL}/ask_agent", json={"text": text})
        print(json.dumps(response.json(), indent=2))
    except Exception as e:
        print(f"🚨 Request failed: {e}")
    print("="*50 + "\n")

# --- Example Test Cases ---
test_api("I'm a 58 year old male and I'm worried about my liver. Can you help check it?")
test_api("What are common symptoms of a fever?")

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-306' coro=<Server.serve() done, defined at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run
    

🚨 Could not get ngrok URL. Make sure the server cell is running. Error: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.
